In [2]:
from google.colab import files
uploaded=files.upload()

Saving top_100_saas_companies_2025.csv to top_100_saas_companies_2025.csv


In [3]:
import pandas as pd

# Load the data
df = pd.read_csv('top_100_saas_companies_2025.csv')

# Step 1: Convert ARR (e.g., "$1.2B") to numeric format
def parse_arr(val):
    if isinstance(val, str):
        val = val.replace('$', '').replace('B', '').strip()
        try:
            return float(val) * 1e9
        except:
            return None
    return None

df['ARR_Num'] = df['ARR'].apply(parse_arr)

# Step 2: Drop rows with missing Industry or ARR
df = df.dropna(subset=['Industry', 'ARR_Num'])

# Step 3: Sort and compute lag (previous company's ARR in the same industry)
df = df.sort_values(by=['Industry', 'ARR_Num'], ascending=[True, False])
df['Previous_ARR'] = df.groupby('Industry')['ARR_Num'].shift(1)

# Step 4: Compute ARR difference with previous company
df['ARR_Difference'] = df['Previous_ARR'] - df['ARR_Num']

# Step 5: Filter where ARR drop exceeds 1 billion USD
steep_drop_df = df[df['ARR_Difference'] > 1e9]

# Step 6: Display results
print("\nCompanies with ARR drop > $1B from previous competitor in their industry:")
print(steep_drop_df[['Company Name', 'Industry', 'ARR', 'ARR_Num', 'Previous_ARR', 'ARR_Difference']])



Companies with ARR drop > $1B from previous competitor in their industry:
   Company Name             Industry     ARR       ARR_Num  Previous_ARR  \
30          Box        Cloud Storage     $1B  1.000000e+09  2.500000e+09   
58  RingCentral       Communications   $2.2B  2.200000e+09  4.100000e+09   
56  CrowdStrike        Cybersecurity   $3.1B  3.100000e+09  7.500000e+09   
22     Palantir       Data Analytics   $2.2B  2.200000e+09  3.700000e+09   
4           SAP  Enterprise Software  $32.5B  3.250000e+10  2.700000e+11   
20       Stripe             Payments    $14B  1.400000e+10  1.970000e+10   

    ARR_Difference  
30    1.500000e+09  
58    1.900000e+09  
56    4.400000e+09  
22    1.500000e+09  
4     2.375000e+11  
20    5.700000e+09  
